In [1]:
import sqlite3
import pandas as pd
from pokemon_showdown_replay_tools.analysis import parse_replay
from tqdm.notebook import tqdm

In [2]:
%%time
con = sqlite3.connect('../replays.regh.db')
cur = con.cursor()
# df = pd.read_sql_query("SELECT * FROM replays LIMIT 10000", con)
df = pd.read_sql_query("SELECT * FROM replays", con)
df = df.set_index('id')
con.close()
print(df.format.value_counts())
df = df[~(df.format == "error")]
df = df[~(df.rating == "null")]
df.rating = df.rating.astype(int)
df = df[df.rating > 1200]
df

format
gen9vgc2024regh       331676
gen9vgc2024reghbo3    167826
error                      2
Name: count, dtype: int64
CPU times: total: 35.9 s
Wall time: 1min 27s


,format,players,log,uploadtime,rating
id,,,,,
gen9vgc2024regh-2235587048,gen9vgc2024regh,"Moralchaussette,lyb0314",|j|☆Moralchaussette\n|j|☆lyb0314\n|t:|17304943...,1.730495e+09,1359
gen9vgc2024regh-2235579839,gen9vgc2024regh,"13dobe,RustyChair13",|j|☆13dobe\n|j|☆RustyChair13\n|t:|1730493513\n...,1.730494e+09,1583
gen9vgc2024regh-2235579943,gen9vgc2024regh,"raylix,Gloopy93",|j|☆raylix\n|j|☆Gloopy93\n|t:|1730493524\n|gam...,1.730494e+09,1323
gen9vgc2024regh-2235579701,gen9vgc2024regh,"liam.2025,647w",|j|☆liam.2025\n|j|☆647w\n|t:|1730493499\n|game...,1.730494e+09,1660
gen9vgc2024regh-2235578159,gen9vgc2024regh,"RustyChair13,liam.2025",|j|☆RustyChair13\n|j|☆liam.2025\n|t:|173049332...,1.730494e+09,1621
...,...,...,...,...,...
gen9vgc2024reghbo3-2171723897,gen9vgc2024reghbo3,"txabo mctxabo,I WasForcedIntoOU",|j|☆txabo mctxabo\n|j|☆I WasForcedIntoOU\n|htm...,1.722564e+09,1203
gen9vgc2024reghbo3-2171675737,gen9vgc2024reghbo3,"gassy george,SplattyDaddy",|j|☆gassy george\n|j|☆SplattyDaddy\n|html|<tab...,1.722558e+09,1206
gen9vgc2024reghbo3-2171648466,gen9vgc2024reghbo3,"natcentre,maremeuatu",|j|☆natcentre\n|j|☆maremeuatu\n|html|<table wi...,1.722555e+09,1237


In [3]:
%%time
con = sqlite3.connect(':memory:')
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS replays (
               id TEXT PRIMARY KEY ON CONFLICT IGNORE,
               format TEXT NOT NULL,
               players TEXT NOT NULL,
               uploadtime INTEGER NOT NULL,
               rating INTEGER)""")
df.loc[:, ['format', 'players', 'uploadtime', 'rating']].to_sql("replays", con, if_exists="append")
cur.execute("""
    CREATE TABLE IF NOT EXISTS appearances (
        id TEXT NOT NULL,
        player TEXT NOT NULL,
        pokemon TEXT NOT NULL,
        won INTEGER NOT NULL,
        FOREIGN KEY(id) REFERENCES replays(id)
        CONSTRAINT one_poke_per_player_per_game UNIQUE(id, player, pokemon) ON CONFLICT IGNORE
    )
""")

df['appearances'] = df.log.apply(parse_replay)

pbar = tqdm(df.itertuples(), total=df.shape[0])
def populate_appearances_table(row):
    data = []
    for appearance in row.appearances['pokemon']:
        data.append([
            row.name,
            appearance['player'],
            appearance['name'],
            1 if appearance['player'] == row.appearances['winner'] else 0,
        ])
    cur.executemany("INSERT INTO appearances VALUES(?, ?, ?, ?)", data)
    pbar.update(1)

df.apply(populate_appearances_table, axis=1)
con.commit()

test_query = """
    WITH marginal as (
        WITH player_appearances as (
            SELECT a1.pokemon as p1, a2.pokemon as p2, a1.player, a1.won as won
            FROM appearances as a1, appearances as a2
            WHERE a1.id = a2.id AND a1.player = a2.player AND p1 < p2
        ) SELECT p1, p2, player, COUNT(*) as appearances, SUM(won) as won
          FROM player_appearances
          GROUP BY p1, p2, player
    ) SELECT p1, p2, COUNT(*) as players, SUM(appearances), SUM(won)
      FROM marginal
      GROUP BY p1, p2
"""
for line in cur.execute("EXPLAIN QUERY PLAN " + test_query).fetchall():
    print(line)
marginal_df = pd.DataFrame(cur.execute(test_query).fetchall())
display(marginal_df)
con.close()

  0%|          | 0/201346 [00:00<?, ?it/s]

(2, 0, 0, 'CO-ROUTINE marginal')
(9, 2, 216, 'SCAN a1')
(11, 2, 47, 'SEARCH a2 USING COVERING INDEX sqlite_autoindex_appearances_1 (id=? AND player=? AND pokemon>?)')
(17, 2, 0, 'USE TEMP B-TREE FOR GROUP BY')
(67, 0, 82, 'SCAN marginal')
(70, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')


,0,1,2,3,4
0,Abomasnow,Alcremie,2,2,0
1,Abomasnow,Ambipom,1,1,1
2,Abomasnow,Amoonguss,12,20,10
3,Abomasnow,Annihilape,26,56,25
4,Abomasnow,Araquanid,5,23,9
...,...,...,...,...,...
37833,Yanma,Yanmega,1,3,1
37834,Yanmega,Zoroark-Hisui,3,3,1
37835,Zangoose,Zebstrika,1,2,0
37836,Zangoose,Zweilous,1,2,0


CPU times: total: 1min 26s
Wall time: 1min 44s


In [4]:
marginal_df = marginal_df.rename({
    0: 'p1',
    1: 'p2',
    2: 'players',
    3: 'appearances',
    4: 'wins',
}, axis=1)
marginal_df['win_pct'] = marginal_df.wins / marginal_df.appearances
display(marginal_df.sort_values(by='win_pct', ascending=False))
mask = marginal_df.appearances >= marginal_df.appearances.quantile(0.9)
display(marginal_df[mask].sort_values(by='win_pct', ascending=False))

,p1,p2,players,appearances,wins,win_pct
37830,Wugtrio,Yanmega,2,2,2,1.0
37808,Weezing,Zebstrika,1,2,2,1.0
37792,Wartortle,Weezing-Galar,1,1,1,1.0
37767,Vivillon-Polar,Whimsicott,1,1,1,1.0
37760,Vivillon-Ocean,Whimsicott,1,2,2,1.0
...,...,...,...,...,...,...
37733,Vivillon,Weezing,1,1,0,0.0
37732,Vivillon,Weavile,1,1,0,0.0
37731,Vivillon,Wartortle,1,1,0,0.0
37729,Vivillon,Volbeat,2,2,0,0.0


,p1,p2,players,appearances,wins,win_pct
6753,Blaziken,Ceruledge,12,85,61,0.717647
35566,Samurott-Hisui,Scizor,13,76,54,0.710526
10389,Cinderace,Talonflame,14,103,70,0.679612
25467,Hydrapple,Maushold,21,86,58,0.674419
24647,Haxorus,Ninetales-Alola,23,95,64,0.673684
...,...,...,...,...,...,...
18333,Farigiraf,Milotic,65,107,32,0.299065
10642,Clefable,Gyarados,46,75,22,0.293333
35392,Sableye,Torkoal,30,72,19,0.263889
28607,Lilligant-Hisui,Rillaboom,41,106,27,0.254717


In [5]:
mask = (marginal_df.p1 == "Archaludon") | (marginal_df.p2 == "Archaludon")
mask &= marginal_df.appearances > 1000
display(marginal_df[mask].sort_values(by='win_pct', ascending=False))

,p1,p2,players,appearances,wins,win_pct
3463,Archaludon,Dondozo,360,1046,549,0.524857
3560,Archaludon,Indeedee,401,1587,825,0.519849
3679,Archaludon,Rillaboom,2340,7242,3572,0.493234
3623,Archaludon,Murkrow,687,2916,1434,0.491770
3406,Archaludon,Basculegion,3019,10008,4905,0.490108
3569,Archaludon,Kingambit,1093,2626,1280,0.487433
3643,Archaludon,Pelipper,6994,31742,15465,0.487209
3468,Archaludon,Dragonite,768,2017,982,0.486862
3717,Archaludon,Sneasler,1204,3479,1687,0.484909
3466,Archaludon,Dragapult,555,1452,704,0.484848


In [6]:
mask = marginal_df.appearances >= marginal_df.appearances.quantile(0.95)
display(marginal_df[mask].sort_values(by='win_pct', ascending=False))

,p1,p2,players,appearances,wins,win_pct
14098,Dondozo,Garganacl,76,301,194,0.644518
27858,Klefki,Sinistcha,73,189,121,0.640212
30008,Mandibuzz,Ninetales-Alola,28,243,149,0.613169
20213,Garchomp,Lycanroc,83,527,320,0.607211
34963,Rillaboom,Sinistcha-Masterpiece,93,201,121,0.601990
...,...,...,...,...,...,...
28861,Lucario,Whimsicott,82,183,65,0.355191
1871,Annihilape,Lilligant-Hisui,100,171,60,0.350877
9773,Charizard,Ursaluna-Bloodmoon,99,212,73,0.344340
26500,Indeedee-F,Palafin-Hero,93,170,55,0.323529


In [21]:
%%time
def find_best_team(pairs_df):
    best = set()
    NUM_BEST = 6
    while len(best) < NUM_BEST  and pairs_df.shape[0] > 0:
        pairs_df = pairs_df.sort_values(by='win_pct', ascending=False)
        best_pair = pairs_df.iloc[0].loc[['p1', 'p2']].values
        if len(set(best_pair) | best) <= NUM_BEST:
            for b in best_pair:
                best.add(b)
        pairs_df = pairs_df.iloc[1:]
        mask = pairs_df.p1.isin(best) | pairs_df.p2.isin(best)
        pairs_df = pairs_df[mask]
        # print(f"Best so far: {best}")
    return best

NO_POKE_MASK = [
    'Weezing',
]

best_num_pairs = 0
best_team_win_expectation = 0
result = ()
for i in tqdm(range(10)):
    quantile = (100 - i) / 100
    mask = marginal_df.appearances >= marginal_df.appearances.quantile(quantile)
    pairs_df = marginal_df[mask]

    no_poke_mask = ~(pairs_df.p1.isin(NO_POKE_MASK) | pairs_df.p2.isin(NO_POKE_MASK))
    best_team = find_best_team(pairs_df[no_poke_mask])
    mask = pairs_df.p1.isin(best_team)
    mask &= pairs_df.p2.isin(best_team)
    num_pairs = pairs_df[mask].shape[0]
    team_win_expectation = sum(pairs_df[mask].appearances * pairs_df[mask].win_pct) / sum(pairs_df[mask].appearances)
    if team_win_expectation > best_team_win_expectation:
        best_team_win_expectation = team_win_expectation
        result = quantile, best_team, pairs_df[mask]

quantile, best_team, pairs_df = result
print(f"Quantile threshold = {quantile}")
print(best_team)
print(f"Num pairs = {pairs_df.shape[0]}")
team_win_expectation = sum(pairs_df.appearances * pairs_df.win_pct) / sum(pairs_df.appearances)
print(f"team_win_expectation={team_win_expectation}")
display(pairs_df.sort_values(by="win_pct", ascending=False))

  0%|          | 0/10 [00:00<?, ?it/s]

Quantile threshold = 0.95
{'Garganacl', 'Glimmora', 'Dondozo', 'Lycanroc', 'Volcarona', 'Tyranitar'}
Num pairs = 12
team_win_expectation=0.5276058631921824


,p1,p2,players,appearances,wins,win_pct
14098,Dondozo,Garganacl,76,301,194,0.644518
20809,Garganacl,Volcarona,75,207,123,0.594203
14297,Dondozo,Tyranitar,211,640,369,0.576562
20618,Garganacl,Glimmora,116,491,277,0.564155
29218,Lycanroc,Tyranitar,458,2547,1384,0.543384
14158,Dondozo,Lycanroc,106,303,164,0.541254
14316,Dondozo,Volcarona,294,745,399,0.535570
14107,Dondozo,Glimmora,751,2872,1538,0.535515
20793,Garganacl,Tyranitar,105,410,213,0.519512
37561,Tyranitar,Volcarona,663,1890,964,0.510053


CPU times: total: 250 ms
Wall time: 502 ms
